In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd ..

/Users/xiaoyangsong/Desktop/RL Controller/Offline-RL-Controller-in-AM


In [3]:
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import os

In [14]:
def extract_single_trajectory(trajectory_id, trajectory_length=8):
    trajectory = []
    for j in range(trajectory_length):
        filename = os.path.join('..', 'RL_Dataset', f'trajectory_00{trajectory_id}', f'layer_{j+1}_data.mat')
        data = loadmat(filename)
        # Extract s, a, r
        ss = data['SS_action'][0][0]
        lp = data['LP_action'][0][0]
        u = np.array(data['uFinal'])
        uAll = data['uAll']
        r = -data['meanDeviation'][0][0]
        # Append
        sar = [u, lp, r]
        print(u.shape, lp.shape, ss.shape, r.shape, lp, ss, r)
        trajectory.append(sar)
    return trajectory

trajectory = extract_single_trajectory(trajectory_id=1, trajectory_length=8)

(1053, 1) () () () 435.21418112695454 730.7743435678826 -0.36122621301829466
(1053, 1) () () () 126.19661470414442 1421.1915508818497 -0.37706539554349666
(1053, 1) () () () 489.385336080055 979.1369092305144 -3.486651536021735
(1053, 1) () () () 504.76113742873747 1475.521122545086 -7.3816908121369105
(1053, 1) () () () 152.88703918942826 1252.1645764000257 -3.3861466576663957
(1053, 1) () () () 513.6671540129037 879.1502090256 -8.104272687881398
(1053, 1) () () () 134.9943433746089 415.3986297303739 -3.9907598469614585
(1053, 1) () () () 517.8947077982968 929.7149581288493 -9.836594728975825


In [16]:
def gather_dataset(id_list, trajectory_length=8):
    dataset = []
    for trajectory_id in id_list:
        traj = extract_single_trajectory(trajectory_id, trajectory_length)
        dataset.append(traj)
    return dataset

id_list = [1, 2, 3, 4, 5]
dataset = gather_dataset(id_list, trajectory_length=8)

(1053, 1) () () () 435.21418112695454 730.7743435678826 -0.36122621301829466
(1053, 1) () () () 126.19661470414442 1421.1915508818497 -0.37706539554349666
(1053, 1) () () () 489.385336080055 979.1369092305144 -3.486651536021735
(1053, 1) () () () 504.76113742873747 1475.521122545086 -7.3816908121369105
(1053, 1) () () () 152.88703918942826 1252.1645764000257 -3.3861466576663957
(1053, 1) () () () 513.6671540129037 879.1502090256 -8.104272687881398
(1053, 1) () () () 134.9943433746089 415.3986297303739 -3.9907598469614585
(1053, 1) () () () 517.8947077982968 929.7149581288493 -9.836594728975825
(1053, 1) () () () 301.77164668602336 581.0718911787624 -0.5544212187870767
(1053, 1) () () () 181.71106387504352 1352.0131111735564 -0.36400716823206797
(1053, 1) () () () 175.8699196795555 1300.2377529680389 -0.18720062542809024
(1053, 1) () () () 312.40573743068705 904.7208555560082 -1.9851228930730056
(1053, 1) () () () 498.11162169650294 580.5579760932429 -6.678266150879865
(1053, 1) () () (

FileNotFoundError: [Errno 2] No such file or directory: '../RL_Dataset/trajectory_005/layer_4_data.mat'